<a href="https://colab.research.google.com/github/viks-iitj/Assignment2-G24AI2004-VivekKumarSrivastava/blob/main/Assignment2_G24AI2004_VivekKumarSrivastava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Indian Institute of Technology Jodhpur
##Fundamentals of Distributed Systems

###Assignment – 2

Total Marks: 20, Submission Deadline: 27 July 2025

###Submitted by : Vivek Kumar Srivastava (Roll- G24AI2004)
Colab Link (Read Only):
https://colab.research.google.com/drive/
1mQ_iLOlu29iC3M96XPHEBk7S6xo7tWGn?usp=sharing

Github Link : https://github.com/viks-iitj/Assignment2-G24AI2004-
VivekKumarSrivastava.git

Colab (Editable), you can get it from Github by click on “Open in Colab icon” to
execute the project:
https://colab.research.google.com/github/viks-iitj/Assignment2-G24AI2004-
VivekKumarSrivastava/blob/main/
Assignment2_G24AI2004_VivekKumarSrivastava.ipynb

##==============================================================================

## Setup environment : Installing necessary dependencies

*   mrjob
*   Hadoop




In [ ]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

--2025-07-28 17:01:23--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz.1’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  23.1MB/s    in 32s     

2025-07-28 17:01:55 (22.0 MB/s) - ‘hadoop-3.3.6.tar.gz.1’ saved [730107476/730107476]



In [ ]:
!tar -xzf hadoop-3.3.6.tar.gz

In [ ]:
!mv hadoop-3.3.6 /usr/local/hadoop

##==============================================================================

#1. Cruiseline Aggregations (5 marks) :
Using cruise.csv, implement an mrjob class that computes, for each Cruise line:

(a) Total number of ships.

(b) Average Tonnage (to two decimals).

(c) Maximum crew size.

(Optional) Use a Combiner for partial aggregation.

### 1.   Load full Data Set :




In [ ]:
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv

--2025-07-28 17:07:49--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘cruise.csv’

cruise.csv          100%[===================>]   8.53K  --.-KB/s    in 0s      

2025-07-28 17:07:49 (77.6 MB/s) - ‘cruise.csv’ saved [8734/8734]



### 2.   Create Sample data from the downloaded file :

In [ ]:
import csv
import random

# Read the cruise.csv file
rows = []
with open("cruise.csv", newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        rows.append(row)

# Choose random 10 samples
sampled_rows = random.sample(rows, 10)

# Save to sample_cruise.csv
with open("sample_cruise.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(sampled_rows)


In [ ]:
!cat sample_cruise.csv

Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
Grand,Princess,15,108.806,26.0,9.51,13.0,41.85,11.1
Adventure,Royal_Caribbean,12,138.0,31.14,10.2,15.57,44.32,11.85
Victory,Carnival,13,101.509,27.58,8.93,13.79,36.81,11.5
Oriana,P&O,18,69.153,18.82,8.53,9.14,36.74,7.94
Dawn,Norwegian,11,90.0,22.4,9.65,11.2,40.18,11.0
Inspiration,Carnival,17,70.367,20.52,8.55,10.2,34.29,9.2
Musica,MSC,7,89.6,25.5,9.61,12.75,35.14,9.87
Constellation,Celebrity,11,91.0,20.32,9.65,9.75,44.78,9.99
Saphire,Princess,9,113.0,26.74,9.51,13.37,42.26,12.38
Oosterdam,Holland_American,10,81.76899999999999,18.48,9.59,9.24,44.25,8.42


### 3. Create mrjob class :

In [ ]:
%%writefile cruise_stats.py
# -----------------------------------------------------------------------------
# Author: Vivek Srivastava
# Date: 2025-07-26
# Description: MRJob script to calculate for each cruise line:
#              - Total number of ships
#              - Average tonnage (rounded to 2 decimals)
#              - Maximum crew size
# -----------------------------------------------------------------------------

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class CruiseLineStats(MRJob):

    # ------------------------------------------------------------
    # Mapper: Parses each line and emits (cruise_line, (1, tonnage, crew))
    # ------------------------------------------------------------
    def mapper(self, _, line):
        try:
            if isinstance(line, bytes):
                line = line.decode('utf-8')

            row = next(csv.reader(io.StringIO(line)))

            if row[0].strip().lower() == "ship_name":
                return

            cruise_line = row[1].strip()
            tonnage = float(row[3].strip())
            crew = float(row[8].strip())

            yield cruise_line, (1, tonnage, crew)

        except:
            return

    # ------------------------------------------------------------
    # Combiner: Partially aggregates count, total tonnage, and max crew per cruise line
    # ------------------------------------------------------------
    def combiner(self, cruise_line, values):
        count, total_tonnage, max_crew = 0, 0.0, 0.0
        for c, t, cr in values:
            count += c
            total_tonnage += t
            max_crew = max(max_crew, cr)
        yield cruise_line, (count, total_tonnage, max_crew)

    # ------------------------------------------------------------
    # Reducer: Final aggregation and formatting output for each cruise line
    # ------------------------------------------------------------
    def reducer(self, cruise_line, values):
        count, total_tonnage, max_crew = 0, 0.0, 0.0
        for c, t, cr in values:
            count += c
            total_tonnage += t
            max_crew = max(max_crew, cr)

        if count > 0:
            avg_tonnage = round(total_tonnage / count, 2)
            result = f"{cruise_line:<25}{count:>6}{avg_tonnage:>15}{max_crew:>12}"
            yield None, result

    # ------------------------------------------------------------
    # Define MapReduce steps using MRStep API
    # ------------------------------------------------------------
    def steps(self):
        return [MRStep(mapper=self.mapper,
                       combiner=self.combiner,
                       reducer=self.reducer)]

# Run the job
if __name__ == '__main__':
    CruiseLineStats.run()


Overwriting cruise_stats.py


### 4. Run Script on Sample Data :

In [ ]:
!python3 cruise_stats.py sample_cruise.csv > sample_cruise.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_stats.root.20250728.170227.437696
Running step 1 of 1...
job output is in /tmp/cruise_stats.root.20250728.170227.437696/output
Streaming final output from /tmp/cruise_stats.root.20250728.170227.437696/output...
Removing temp directory /tmp/cruise_stats.root.20250728.170227.437696...


### 5. Display Sample Test Result :

In [ ]:
# Show the header once
print(f"{'Cruise Line':<25}{'Ships':>6}{'Avg Tonnage':>15}{'Max Crew':>12}")
print("-" * 60)

# Read and clean output lines from the file
with open("sample_cruise.txt") as f:
    for line in f:
        line = line.strip()
        if "\t" in line:
            _, value = line.split("\t", 1)
            print(value.strip('"'))


Cruise Line               Ships    Avg Tonnage    Max Crew
------------------------------------------------------------
MSC                           1           89.6        9.87
Norwegian                     1           90.0        11.0
P&O                           1          69.15        7.94
Princess                      2          110.9       12.38
Royal_Caribbean               1          138.0       11.85
Carnival                      2          85.94        11.5
Celebrity                     1           91.0        9.99
Holland_American              1          81.77        8.42


### 6. Run Scripton on Full Dataset :

In [ ]:
!python3 cruise_stats.py cruise.csv > cruise.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_stats.root.20250728.170228.101517
Running step 1 of 1...
job output is in /tmp/cruise_stats.root.20250728.170228.101517/output
Streaming final output from /tmp/cruise_stats.root.20250728.170228.101517/output...
Removing temp directory /tmp/cruise_stats.root.20250728.170228.101517...


### 7. Display Result for Full Data Set (Final Output) :

In [ ]:
# Show the header once
print(f"{'Cruise Line':<25}{'Ships':>6}{'Avg Tonnage':>15}{'Max Crew':>12}")
print("-" * 60)

# Read and clean output lines from the file
with open("cruise.txt") as f:
    for line in f:
        line = line.strip()
        if "\t" in line:
            _, value = line.split("\t", 1)
            print(value.strip('"'))


Cruise Line               Ships    Avg Tonnage    Max Crew
------------------------------------------------------------
Cunard                        3         103.91       12.53
Disney                        2          83.17        9.45
Holland_American             14           60.5        8.42
MSC                           8          63.77       13.13
Norwegian                    13          63.72        13.0
Oceania                       3          30.28         4.0
Orient                        1          22.08         3.5
P&O                           6          77.86        12.2
Princess                     17          87.54       12.38
Regent_Seven_Seas             5          32.14        4.47
Royal_Caribbean              23         107.01        21.0
Azamara                       2          30.28        3.55
Carnival                     22          84.65        19.1
Celebrity                    10          76.16        9.99
Costa                        11           71.1        

##==============================================================================

#2. Company Churn Rate (5 marks)
From customer churn.csv, create a MultiStepJob:

Step 1: Mapper emits (Company, TOTAL) and (Company, CHURNED) where Churn==1.

Step 2: Reducer computes churn rate=CHURNED/TOTAL , outputting four-decimal floats.

Use a small VIP companies.txt in the distributed cache to restrict to listed companies.
Provide a sample file with at least three names.

### 1.   Load full Data Set :

In [ ]:
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv

--2025-07-28 17:08:46--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115479 (113K) [text/plain]
Saving to: ‘customer_churn.csv’

customer_churn.csv  100%[===================>] 112.77K  --.-KB/s    in 0.02s   

2025-07-28 17:08:47 (4.53 MB/s) - ‘customer_churn.csv’ saved [115479/115479]



### 2. Create Sample data from the downloaded file :

In [ ]:
import csv
import random

# Read the customer_churn.csv file
rows = []
with open("customer_churn.csv", newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        rows.append(row)

# Choose random 10 samples
sampled_rows = random.sample(rows, 10)

# Save to sample_customer_churn.csv
with open("sample_customer_churn.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(sampled_rows)

In [ ]:
!cat sample_customer_churn.csv

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
Lawrence Gordon,44.0,8817.22,1,6.53,10.0,2009-09-23 09:35:38,"4041 Wise River Suite 828 West Todd, AR 08725-4767","Austin, Roberts and Cox",0
Meghan Jackson,46.0,9844.79,0,3.57,11.0,2010-01-03 10:44:43,"6574 Travis Extensions New Casey, RI 40982",Phillips-Peters,0
Kenneth Wilkerson,42.0,11670.32,0,5.51,7.0,2014-05-08 21:37:57,Unit 8005 Box 5636 DPO AA 90315-7695,"Aguilar, Kelly and Fuentes",0
Daniel Barton,44.0,9855.18,0,6.94,7.0,2011-06-10 20:04:28,"551 Davis Crescent New Aliciaport, TX 94552-1600",Davidson Group,0
Cheryl Bruce,44.0,9613.84,1,4.38,8.0,2014-06-18 19:25:33,"PSC 5054, Box 2004 APO AP 20629","Oconnor, Hernandez and Bailey",0
Laura Turner,42.0,8934.99,1,6.21,8.0,2010-10-26 19:01:20,"46278 Austin Views Apt. 675 Port Paulfurt, KS 08809-6244","Johnson, Myers and Martinez",0
Devin Wilson,46.0,11222.48,0,4.92,9.0,2011-02-18 18:11:37,"97426 Carrie Stravenue East Cassandrafort, WV 23196",

### 3. Creating VIP_companies text file :

In [ ]:
%%writefile VIP_companies.txt
Cooke Ltd
Reynolds-Sheppard
Austin Ltd
Smith Inc
Phillips-Peters

Overwriting VIP_companies.txt


### 4. Create mrjob class :
class with a MultiStepJob to calculate the churn rate for each company, using the distributed cache to filter for VIP companies.

In [ ]:
%%writefile churn_rate.py
# -----------------------------------------------------------------------------
# Author: Vivek Srivastava
# Date: 2025-07-26
# Description: MRJob script to compute churn rate per company using a two-step
#              process restricted to VIP companies listed in a side file.
#              Output: Company → Churn Rate (4 decimal places)
# -----------------------------------------------------------------------------

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class CompanyChurnRate(MRJob):

    # ------------------------------------------------------------
    # Configure command-line argument to accept VIP file
    # ------------------------------------------------------------
    def configure_args(self):
        super().configure_args()
        self.add_file_arg('--vipfile', help='Path to VIP companies file')

    # ------------------------------------------------------------
    # Mapper Init: Load VIP companies from file into a set
    # ------------------------------------------------------------
    def mapper_init(self):
        self.vip_companies = set()
        if self.options.vipfile:
            with open(self.options.vipfile, 'r') as f:
                self.vip_companies = {line.strip() for line in f}

    # ------------------------------------------------------------
    # Mapper: Emits (company, 'TOTAL') and (company, 'CHURNED')
    # for VIP companies only, based on churn flag
    # ------------------------------------------------------------
    def mapper(self, _, line):
        try:
            row = next(csv.reader([line], delimiter=","))

            # Skip header row
            if row[0].strip().lower() == "names":
                return

            company = row[8].strip().strip('"')
            churn = row[9].strip()

            if company in self.vip_companies:
                yield company, "TOTAL"
                if churn == "1":
                    yield company, "CHURNED"
        except:
            pass

    # ------------------------------------------------------------
    # Reducer: Calculates churn rate = CHURNED / TOTAL
    # ------------------------------------------------------------
    def reducer(self, company, values):
        total = churned = 0
        for v in values:
            if v == "TOTAL":
                total += 1
            elif v == "CHURNED":
                churned += 1

        if total > 0:
            churn_rate = churned / total
            yield company, f"{churn_rate:.4f}"

    # ------------------------------------------------------------
    # MapReduce Steps API
    # ------------------------------------------------------------
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                reducer=self.reducer
            )
        ]

# Run the job
if __name__ == '__main__':
    CompanyChurnRate.run()


Overwriting churn_rate.py


### 5. Run Script on Sample Data :

In [ ]:
!python3 churn_rate.py sample_customer_churn.csv --vipfile VIP_companies.txt > sample_churn_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/churn_rate.root.20250728.173834.250205
Running step 1 of 1...
job output is in /tmp/churn_rate.root.20250728.173834.250205/output
Streaming final output from /tmp/churn_rate.root.20250728.173834.250205/output...
Removing temp directory /tmp/churn_rate.root.20250728.173834.250205...


### 6. Display Sample Test Result :

In [ ]:
print(f"{'Company':<30}{'Churn Rate':>12}")
print("-" * 42)

with open("sample_churn_output.txt") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) == 2:
            company = parts[0].strip('"')
            rate = parts[1].strip()
            print(f"{company:<30}{rate:>12}")


Company                         Churn Rate
------------------------------------------
Phillips-Peters                        0.0
Smith Inc                              1.0
Cooke Ltd                              1.0


### 7. Run Scripton on Full Dataset :

In [ ]:
!python3 churn_rate.py customer_churn.csv --vipfile VIP_companies.txt > customer_churn.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/churn_rate.root.20250728.173841.967681
Running step 1 of 1...
job output is in /tmp/churn_rate.root.20250728.173841.967681/output
Streaming final output from /tmp/churn_rate.root.20250728.173841.967681/output...
Removing temp directory /tmp/churn_rate.root.20250728.173841.967681...


### 8. Display Result for Full Data Set (Final Output) :

In [ ]:
print(f"{'Company':<30}{'Churn Rate':>12}")
print("-" * 42)
with open("customer_churn.txt") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) == 2:
            company = parts[0].strip('"')
            rate = parts[1].strip()
            print(f"{company:<30}{rate:>12}")


Company                         Churn Rate
------------------------------------------
Phillips-Peters                        0.0
Reynolds-Sheppard                      1.0
Smith Inc                              1.0
Austin Ltd                             0.0
Cooke Ltd                              1.0


##==============================================================================

# 3. State-wise Spending (5 marks)
From e-com customer.csv, extract the two-letter state code from the Address field.

Then:

• Mapper parses the state.

• Reducer sums Yearly Amount Spent per state.

• Output the top 5 states by total spending.

### 1. Load full Data Set :

In [ ]:
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv

--2025-07-28 17:09:39--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86871 (85K) [text/plain]
Saving to: ‘e-com_customer.csv’

e-com_customer.csv  100%[===================>]  84.83K  --.-KB/s    in 0.02s   

2025-07-28 17:09:40 (3.50 MB/s) - ‘e-com_customer.csv’ saved [86871/86871]



### 2. Create Sample data from the downloaded file :

In [ ]:
import csv
import random

# Read the e-com_customer.csv file
rows = []
with open("e-com_customer.csv", newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        rows.append(row)

# Choose random 10 samples
sampled_rows = random.sample(rows, 10)

# Save to sample_e-com_customer.csv
with open("sample_e-com_customer.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(sampled_rows)

In [ ]:
!cat sample_e-com_customer.csv

Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
brian51@cook.info,"0508 Terrance CliffRebekahtown, NC 48724-8308",SaddleBrown,31.51712180250623,10.745188554182882,38.79123468689964,1.4288238768282668,275.9184206503857
tracy79@wheeler.net,"513 Moore Crescent Apt. 416Amandaberg, GU 03655-2219",Magenta,32.2463499961189,11.305551430414742,37.133126759376324,1.7073897286757314,327.37795258965207
esmith@gmail.com,USS JohnsonFPO AE 35617-5384,OldLace,34.18382052058911,13.349912944366533,37.827394227862136,4.252006105620936,614.7153338263417
ethomas@yahoo.com,"1279 Douglas MountainsPort Christopherberg, RI 32811-8325",LightGreen,33.69489765096749,11.202669899735568,35.49396408005128,4.015986640445782,505.11963752820367
morganorozco@hotmail.com,"0001 Mack MillNorth Jennifer, NE 42021-5936",LightPink,30.4925366965402,11.562936246652605,35.97656497174036,1.4816166268553612,282.4712457199145
riverarebecca@gmail.com,"1414 David Throughway

### 3. Create mrjob class :

In [ ]:
%%writefile top_states_spending.py
# -----------------------------------------------------------------------------
# Author: Vivek Srivastava
# Date: 2025-07-27
# Description: MRJob script to calculate top 5 U.S. states by total yearly
#              customer spending from e-commerce data.
# -----------------------------------------------------------------------------

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class TopStatesSpending(MRJob):

    # ------------------------------------------------------------
    # Mapper: Extracts the state code and spending amount.
    # Emits: (state_code, yearly_amount_spent)
    # ------------------------------------------------------------
    def mapper(self, _, line):
        try:
            # Use csv.reader to handle quoted fields with commas
            row = next(csv.reader(io.StringIO(line)))

            # Skip header row
            if row[0].strip().lower() == "email":
                return

            address = row[1].strip()
            amount = float(row[7].strip())

            # Extract the two-letter state code from the address
            parts = address.split()
            if len(parts) >= 2:
                state = parts[-2].strip()
                if len(state) == 2 and state.isalpha():
                    yield state.upper(), amount
        except:
            pass

    # ------------------------------------------------------------
    # Reducer: Sums all amounts spent for each state.
    # Emits: (None, (total_amount, state))
    # ------------------------------------------------------------
    def reducer(self, state, amounts):
        yield None, (sum(amounts), state)

    # ------------------------------------------------------------
    # Final Reducer: Sorts all (total, state) pairs and emits
    # the top 5 states with the highest total spending.
    # ------------------------------------------------------------
    def reducer_top5(self, _, state_amounts):
        top5 = sorted(state_amounts, reverse=True)[:5]
        for total, state in top5:
            yield state, round(total, 4)

    # ------------------------------------------------------------
    # Defines the MapReduce pipeline using MRStep API
    # ------------------------------------------------------------
    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.reducer_top5)
        ]

# Run the job
if __name__ == '__main__':
    TopStatesSpending.run()


Overwriting top_states_spending.py


### 4. Run Script on Sample Data :

In [ ]:
!python3 top_states_spending.py sample_e-com_customer.csv > sample_e-com_customer.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/top_states_spending.root.20250728.170230.648356
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/top_states_spending.root.20250728.170230.648356/output
Streaming final output from /tmp/top_states_spending.root.20250728.170230.648356/output...
Removing temp directory /tmp/top_states_spending.root.20250728.170230.648356...


### 5. Display Sample Test Result :

In [ ]:
print(f"{'State':<10}{'Total Spending'}")
print("-" * 30)

with open("sample_e-com_customer.txt") as f:
    for line in f:
        if "\t" in line:
            state, amount = line.strip().split("\t")
            print(f"{state:<10}{amount}")


State     Total Spending
------------------------------
"AE"      1222.9872
"OR"      630.4228
"OH"      581.8523
"TX"      537.7732
"RI"      505.1196


### 6. Run Scripton on Full Dataset :

In [ ]:
!python3 top_states_spending.py e-com_customer.csv > e-com_customer.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/top_states_spending.root.20250728.170231.077373
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/top_states_spending.root.20250728.170231.077373/output
Streaming final output from /tmp/top_states_spending.root.20250728.170231.077373/output...
Removing temp directory /tmp/top_states_spending.root.20250728.170231.077373...


### 7. Display Result for Full Data Set (Final Output) :

In [ ]:
print(f"{'State':<10}{'Total Spending'}")
print("-" * 30)
with open("e-com_customer.txt") as f:
    for line in f:
        if "\t" in line:
            state, amount = line.strip().split("\t")
            print(f"{state:<10}{amount}")


State     Total Spending
------------------------------
"AE"      8686.8303
"AA"      8404.697
"SC"      6820.2978
"DE"      6644.9813
"MO"      6402.5697


##==============================================================================

# 4. Two-step Ship Filter & Median Length (5 marks)
On cruise.csv, implement a two-step mrjob pipeline:

Step 1: Filter ships with passenger density > 35.0; emit ⟨Cruise line, length⟩.

Step 2: Compute the median of the lengths per Cruise line, handling even/odd counts correctly.

Use the steps() API and output medians to two decimals.

### 1. Load full Data Set (creating duplicate as we have already added the data set as a part of Question1, will use the same data set for this question) :

In [ ]:
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv

--2025-07-28 17:11:37--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘cruise.csv.1’

cruise.csv.1        100%[===================>]   8.53K  --.-KB/s    in 0.001s  

2025-07-28 17:11:37 (14.8 MB/s) - ‘cruise.csv.1’ saved [8734/8734]



### 2. Create Sample data from the downloaded file :

In [ ]:
import csv
import random

# Read the cruise.csv file
rows = []
with open("cruise.csv", newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        rows.append(row)

# Choose random 10 samples
sampled_rows = random.sample(rows, 10)

# Save to sample_cruise-1.csv
with open("sample_cruise-1.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(sampled_rows)

In [ ]:
!cat sample_cruise-1.csv

Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
Spirit,Carnival,12,88.5,21.24,9.63,10.56,41.67,10.29
Navigator,Regent_Seven_Seas,14,33.0,4.9,5.6,2.45,67.35,3.24
Golden,Princess,12,108.865,27.58,9.51,13.0,39.47,11.0
Solstice,Celebrity,5,122.0,28.5,10.33,6.87,34.57,6.7
Rhapsody,Royal_Caribbean,16,78.491,24.35,9.15,10.0,32.23,7.65
Summit,Celebrity,12,91.0,20.32,9.65,9.75,44.78,9.99
Enchantment,Royal_Caribbean,16,74.137,19.5,9.16,9.75,38.02,7.6
Monarch,Royal_Caribbean,22,73.941,27.44,8.8,11.77,30.94,8.22
ExplorerII,Regent_Seven_Seas,27,12.5,3.94,4.36,0.88,31.73,1.46
Sea,Norwegian,25,42.0,15.04,7.08,7.52,27.93,6.3


### 3. Create mrjob class :

In [ ]:
%%writefile cruise_median_length.py
# -----------------------------------------------------------------------------
# Author: Vivek Srivastava
# Date: 2025-07-27
# Description: MRJob script to compute median ship length per cruise line
#              for ships with passenger density > 35.0
# -----------------------------------------------------------------------------

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class CruiseMedianLength(MRJob):

    # ------------------------------------------------------------
    # Step 1 Mapper: Filter ships with passenger_density > 35.0
    # Emits: (Cruise_line, length)
    # ------------------------------------------------------------
    def mapper_filter_density(self, _, line):
        try:
            row = next(csv.reader(io.StringIO(line)))

            # Skip header row
            if row[0].strip().lower() == "ship_name":
                return

            cruise_line = row[1].strip()
            length = float(row[5].strip())
            density = float(row[7].strip())

            if density > 35.0:
                yield cruise_line, length
        except:
            pass

    # ------------------------------------------------------------
    # Step 1 Reducer: Collect all lengths for each cruise line
    # Computes the median of the lengths
    # ------------------------------------------------------------
    def reducer_group_lengths(self, cruise_line, lengths):
        sorted_lengths = sorted(lengths)
        n = len(sorted_lengths)

        if n == 0:
            return

        # Compute median for odd or even number of ships
        if n % 2 == 1:
            median = sorted_lengths[n // 2]
        else:
            median = (sorted_lengths[n // 2 - 1] + sorted_lengths[n // 2]) / 2.0

        # Emit cruise line with median length rounded to 2 decimals
        yield cruise_line, round(median, 2)

    # ------------------------------------------------------------
    # Define MapReduce pipeline using MRStep
    # ------------------------------------------------------------
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_filter_density,
                reducer=self.reducer_group_lengths
            )
        ]

# Run the job
if __name__ == '__main__':
    CruiseMedianLength.run()


Overwriting cruise_median_length.py


### 4. Run Script on Sample Data :

In [ ]:
!python3 cruise_median_length.py sample_cruise-1.csv > sample_cruise-1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_median_length.root.20250728.170231.744616
Running step 1 of 1...
job output is in /tmp/cruise_median_length.root.20250728.170231.744616/output
Streaming final output from /tmp/cruise_median_length.root.20250728.170231.744616/output...
Removing temp directory /tmp/cruise_median_length.root.20250728.170231.744616...


### 5. Display Sample Test Result :

In [ ]:
print(f"{'Cruise Line':<25}{'Median Length'}")
print("-" * 40)

with open("sample_cruise-1.txt") as f:
    for line in f:
        if "\t" in line:
            cruise_line, median = line.strip().split("\t")
            print(f"{cruise_line:<25}{median}")


Cruise Line              Median Length
----------------------------------------
"Princess"               9.51
"Regent_Seven_Seas"      5.6
"Royal_Caribbean"        9.16
"Carnival"               9.63
"Celebrity"              9.65


### 6. Run Scripton on Full Dataset :

In [ ]:
!python3 cruise_median_length.py cruise.csv > cruise-1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_median_length.root.20250728.170232.168922
Running step 1 of 1...
job output is in /tmp/cruise_median_length.root.20250728.170232.168922/output
Streaming final output from /tmp/cruise_median_length.root.20250728.170232.168922/output...
Removing temp directory /tmp/cruise_median_length.root.20250728.170232.168922...


In [ ]:
print(f"{'Cruise Line':<25}{'Median Length'}")
print("-" * 40)

with open("cruise-1.txt") as f:
    for line in f:
        if "\t" in line:
            cruise_line, median = line.strip().split("\t")
            print(f"{cruise_line:<25}{median}")


Cruise Line              Median Length
----------------------------------------
"Disney"                 9.64
"Holland_American"       7.77
"MSC"                    8.23
"Norwegian"              9.0
"Oceania"                5.94
"P&O"                    8.56
"Princess"               9.51
"Regent_Seven_Seas"      6.15
"Royal_Caribbean"        10.2
"Azamara"                5.94
"Carnival"               9.52
"Celebrity"              9.65
"Costa"                  8.28
"Crystal"                7.86
"Cunard"                 9.64
"Seabourn"               4.4
"Silversea"              5.55
"Star"                   2.8
"Windstar"               6.17
